In [1]:
from common import load_data, files_rename

files_rename('src')
data, labels = load_data('src', 50)
print('Size of Data:', len(data))
print('Size of Label:', len(labels))

Size of Data: 50
Size of Label: 50


In [2]:
import preprocessor as pp
import numpy as np

dataset = [] # dataset
label = [] # label
for (index, image) in enumerate(data):
    normalized = pp.hist_normalize(image)
    houghed = pp.hough_circling(normalized)
    if houghed is not None:
        hog_img = pp.hog_descriptor(houghed)
        dataset.append(hog_img)
        label.append(labels[index])

print('dataset size:', len(dataset))
print('label size:', len(label))

dataset size: 45
label size: 45


In [3]:
from tensorflow.keras.utils import to_categorical

X = np.array(dataset)
X = X.reshape(*X.shape, 1)
label_set = set(label)
classes = { val: key + 1 for (key, val) in enumerate(label_set)}
Y = np.fromiter([classes[y] for y in label], dtype=np.int)
Y = to_categorical(Y)
dense = len(Y[0])
dense

4

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train.shape

(40, 512, 256, 1)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

# initiate model
model = Sequential()

# add model layers
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(512, 256, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(Flatten())
model.add(Dense(dense, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 509, 253, 32)      544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 254, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 251, 123, 64)      32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 125, 61, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 58, 128)      131200    
_________________________________________________________________
flatten (Flatten)            (None, 905728)            0         
_________________________________________________________________
dense (Dense)                (None, 4)                 3

In [6]:
# compile model using accuracy to measure the performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Epoch 1/3
2/2 [==============================] - 4s 916ms/step - loss: 1.2721 - accuracy: 0.1750 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 2/3
2/2 [==============================] - 3s 833ms/step - loss: 1.0053 - accuracy: 0.8250 - val_loss: 0.3938 - val_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 3s 792ms/step - loss: 0.4353 - accuracy: 0.8750 - val_loss: 0.2942 - val_accuracy: 1.0000


In [8]:
model.save('saved_models/my_model')

INFO:tensorflow:Assets written to: saved_models/my_model\assets
